In [1]:
import os
os.remove('snorkel.db')

In [2]:
from snorkel import SnorkelSession
session = SnorkelSession()

import os, sys
sys.path.append(os.environ['SNORKELHOME'] + '/tutorials/tables/')

In [3]:
import os
from snorkel.parser import CorpusParser
from snorkel.parser import HTMLParser
from snorkel.parser import OmniParser

doc_parser = HTMLParser(path=os.environ['SNORKELHOME'] + '/tutorials/tables/sandbox/diseases.xhtml')
context_parser = OmniParser()
cp = CorpusParser(doc_parser, context_parser, max_docs=1)

In [4]:
%time corpus = cp.parse_corpus(name='Sandbox', session=session)
print "%d Phrases in %s" % (
    len([phrase for doc in corpus.documents for phrase in doc.phrases]), corpus)

[========================================] 100%

CPU times: user 296 ms, sys: 22.7 ms, total: 319 ms
Wall time: 529 ms
31 Phrases in Corpus (Sandbox)


In [5]:
for cell in corpus.documents[0].cells: print cell.text[:10]

<th hobbie
<th day="M
<th>Year</
<th>Polio 
<td style=
<td style=
<th>I don'
<td>New Yo
<td></td>
<th>Scurvy
<td>Annapo
<td>1901</
<th>Proble
<th>Cause<
<th>Cost</
<th>Arthri
<td>Pokemo
<td>Free</
<th>Yellow
<td>Unicor
<td>$17.75
<th>Hypoch
<td>Fear</
<td>$100</


In [6]:
from snorkel.lf_helpers import _infer_cell, _get_aligned_cells

root_phrase = corpus.documents[0].tables[0].cells[7].phrases[0]
print root_phrase

infer = True
axis = 'row'
other_axis = 'col_num'
print [cell.text[:8] for cell in _get_aligned_cells(root_phrase.cell, axis, infer=infer)]
print [phrase.text for cell in _get_aligned_cells(root_phrase.cell, axis, infer=infer) for phrase in cell.phrases]
# print [phrase.text for cell in _get_aligned_cells(root_phrase.cell, axis, infer=False) for phrase in _infer_cell(cell, other_axis).phrases]

Phrase(Doc: diseases, Table: 0, Row: 2, Col: 1, Position: 0, Text: New York)
[u'<th>I do', u'<td styl']
[u"I don't like Chicken  Pox  or pizza.", u'Shingles is also bad.', u'% 2001']


In [7]:
# for phrase in [phrase for doc in corpus.documents for phrase in doc.phrases]:
#     print phrase

In [8]:
(a,b,c) = (2,(3 if True else 5) ,4)

In [9]:
from snorkel.models import candidate_subclass

# Year = candidate_subclass('Year', ['year'])
# Temp = candidate_subclass('Temp', ['temp'])
# Disease = candidate_subclass('Disease', ['disease'])
# Part = candidate_subclass('Part', ['part'])
Disease_Year = candidate_subclass('Disease_Year', ['disease','year'])

In [10]:
from snorkel.candidates import OmniNgrams
from hardware_utils import OmniNgramsPart, OmniNgramsTemp

omni_ngrams = OmniNgrams(n_max=3, split_tokens=None)
# omni_part = OmniNgramsPart(n_max=3, split_tokens=None)
# omni_temp = OmniNgramsTemp(n_max=3, split_tokens=None)

In [11]:
diseases = ['viruses', 'coughs', 'colds', 'brain', 'cancer', 'shingles', 'warts',
              'brain cancer', 'disease', 'location', 'polio', 'chicken pox', 
              'bubonic plague', 'plague', 'scurvy', 'infectious diseases', 
              'arthritis', 'yellow fever', 'fever', 'hypochondria', 'pneumonia']
print "Loaded %d diseases." % len(diseases)

Loaded 21 diseases.


In [12]:
from snorkel.matchers import DictionaryMatch, RegexMatchEach, RegexMatchSpan, DateMatcher

disease_matcher = DictionaryMatch(d=diseases, ignore_case=True)
year_matcher = DateMatcher()
# part_matcher = RegexMatchEach(rgx='BC.*')
# temp_matcher = RegexMatchSpan(rgx=r'-[5-7][05]')

In [13]:
from snorkel.candidates import CandidateExtractor

# ce = CandidateExtractor(Part, [omni_part], [part_matcher])
# ce = CandidateExtractor(Disease, [omni_ngrams], [disease_matcher])
# ce = CandidateExtractor(Temp, [omni_temp], [temp_matcher])
ce = CandidateExtractor(Disease_Year, 
                        [omni_ngrams, omni_ngrams], 
                        [disease_matcher, year_matcher])

In [14]:
%time candidates = ce.extract(corpus.documents, 'Sandbox Candidates', session)
print "%s contains %d Candidates" % (candidates, len(candidates))

[========================================] 100%

CPU times: user 105 ms, sys: 7.21 ms, total: 112 ms
Wall time: 112 ms
Candidate Set (Sandbox Candidates) contains 34 Candidates


In [15]:
# for c in candidates: print c

In [16]:
# for c in candidates:
#     print c
c = candidates[14]
print c
print c.disease.parent.document
print c.disease.parent.table
# print c.disease.parent.row
# print c.disease.parent.col
print c.disease.parent.cell
print c.disease.parent

Disease_Year(Span("Location", parent=50, chars=[0,7], words=[0,0]), Span("1901", parent=60, chars=[0,3], words=[0,0]))
Document diseases
Table(Doc: diseases, Position: 0)
Cell(Doc: diseases, Table: 0, Row: 0, Col: 1)
Phrase(Doc: diseases, Table: 0, Row: 0, Col: 1, Position: 0, Text: Location)


In [17]:
from snorkel.models import CandidateSet
train = CandidateSet(name='train', candidates=candidates[:25])
dev = CandidateSet(name='test', candidates=candidates[25:])
print len(train)
print train[0]
print len(dev)
print dev[0]

25
Disease_Year(Span("Brain Cancer", parent=3, chars=[0,11], words=[0,1]), Span("1901", parent=60, chars=[0,3], words=[0,0]))
9
Disease_Year(Span("Infectious diseases", parent=15, chars=[9,27], words=[3,4]), Span("2001", parent=54, chars=[2,5], words=[1,1]))


In [18]:
print len(train)

25


In [19]:
from snorkel.annotations import FeatureManager

feature_manager = FeatureManager()
%time F_train = feature_manager.create(session, train, 'Sandbox Features')
F_train

# from snorkel.features import get_span_feats
# c = candidates[10]
# %prun for feat in get_span_feats(c): print feat

[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 2.77 s, sys: 27.9 ms, total: 2.79 s
Wall time: 2.84 s


<25x538 sparse matrix of type '<type 'numpy.float64'>'
	with 2444 stored elements in Compressed Sparse Row format>

In [20]:
other_manager = FeatureManager()
%time G_train = feature_manager.create(session, dev, 'Other Features')
G_train

[========================================] 100%

Loading sparse Feature matrix...
CPU times: user 969 ms, sys: 17.3 ms, total: 986 ms
Wall time: 1.05 s


<9x251 sparse matrix of type '<type 'numpy.float64'>'
	with 761 stored elements in Compressed Sparse Row format>

In [21]:
# F_train + G_train

In [22]:
c = candidates[25]
print c
print c.disease.parent.row_num
print c.disease.parent.row
print c.disease.parent.col_num
print c.disease.parent.col

Disease_Year(Span("Infectious diseases", parent=15, chars=[9,27], words=[3,4]), Span("2001", parent=54, chars=[2,5], words=[1,1]))
None
None
None
None


In [23]:
from snorkel.utils import get_keys_by_candidate

print candidates[25]
print "Limited number of features being shown:"
for f in get_keys_by_candidate(F_train, F_train.get_candidate(4))[:10]: print f

Disease_Year(Span("Infectious diseases", parent=15, chars=[9,27], words=[3,4]), Span("2001", parent=54, chars=[2,5], words=[1,1]))
Limited number of features being shown:
AnnotationKey (TAB_e1_STARTS_WITH_CAPITAL)
AnnotationKey (TAB_e1_HTML_ANC_TAG_[html])
AnnotationKey (TAB_e1_HTML_ANC_TAG_[body])
AnnotationKey (TAB_e2_WORD_SEQ_[1901])
AnnotationKey (TAB_e2_LEMMA_SEQ_[1901])
AnnotationKey (TAB_e2_POS_SEQ_[CD])
AnnotationKey (TAB_e2_DEP_SEQ_[ROOT])
AnnotationKey (TAB_e2_W_LEFT_1_[_NUMBER])
AnnotationKey (TAB_e2_W_LEFT_POS_1_[CD])
AnnotationKey (TAB_e2_LENGTH_1)


In [24]:
# c = candidates[0]
# print c
# print c.year
# print c.year.parent
# print c.year.char_start
# print c.year.char_end
# print c.year.parent.words
# print c.year.parent.char_offsets
# print c.year.parent.lemmas
# print c.year.parent.pos_tags
# print c.year.parent.ner_tags
# print c.year.parent.dep_parents
# print c.year.parent.dep_labels
# print c.year.get_word_start()
# print c.year.get_word_end()
# # print get_row_ngrams(c.year)
# # print get_row_ngrams(c.year, infer=True)
# %time for i in range(1000): _get_aligned_cells((c.year).parent.cell, 'col', infer=True)

In [25]:
# from snorkel.models import Span, ImplicitSpan, TemporarySpan

# print isinstance(c.year, TemporarySpan)

In [26]:
# from snorkel.lf_helpers import get_between_ngrams, get_left_ngrams, get_right_ngrams
# from snorkel.lf_helpers import contains_token, contains_regex
# from snorkel.lf_helpers import get_phrase_ngrams, get_cell_ngrams, get_neighbor_cell_ngrams
# from snorkel.lf_helpers import get_row_ngrams, get_col_ngrams, get_aligned_ngrams
# from snorkel.lf_helpers import same_document, same_table, same_cell, same_phrase
# from snorkel.lf_helpers import _get_aligned_cells, _get_nonempty_cell

In [27]:
# print get_left_ngrams(c.disease)
# print get_right_ngrams(c.disease)
# print contains_token(c, 'plague')
# print contains_regex(c, r'pla')
# print same_document(c)
# print same_table(c)
# print same_cell(c)
# print same_phrase(c)
# print get_phrase_ngrams(c.disease, n_min=1, n_max=3, case_sensitive=True)
# print get_cell_ngrams(c.disease, attrib='pos_tags')
# print get_neighbor_cell_ngrams(c.disease, dist=2, directions=True)
# print get_row_ngrams(c.disease)
# print get_col_ngrams(c.disease)
# print get_aligned_ngrams(c.disease)
# print get_aligned_ngrams(c.disease, infer=True)

In [28]:
# from hardware_utils import expand_implicit_text 

# for part in expand_implicit_text(''.join(['BC856/857/858', '/', '859/860'])): print part

In [29]:
# def foo(n):
#     phrase = 'repeat me'
#     pmul = phrase * n
#     pjoi = ''.join([phrase for x in xrange(n)])
#     pinc = ''
#     for x in xrange(n):
#         pinc += phrase
#     del pmul, pjoi, pinc

In [30]:
# %lprun -f foo foo(1000)

In [31]:
# from hardware_utils import get_gold_dict

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# gold_dict = get_gold_dict(filename, 'stg_temp_min')

In [32]:
# print len(gold_dict.values())
# print gold_dict.values().count(1)

In [33]:
# from hardware_utils import count_hardware_labels

# filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
# %time (certain, maybe) = count_hardware_labels(loader, candidates, filename, attrib='stg_temp_min', attrib_class='temp')

In [34]:
%time 
for i in range(10000): 
    for j in range(100):
        1 + 1

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs
